In [620]:
import pandas as pd
import numpy as np

# data = pd.read_excel('../data/master.xlsx')
# data = pd.read_excel('../data/master.xlsx').sample(1000)
data = {'A': [3, 3, 2], 'B': [3, 3, 4], 'C': [7, 7, 7]}
data = pd.DataFrame(data)

minSup = 1
numDims = len(data.columns)
outputRec = pd.DataFrame({
    "Combination": [],
    "Count": []
})


In [621]:
data.columns

Index(['A', 'B', 'C'], dtype='object')

In [622]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [623]:
# unique_counts = data.nunique()
# col_dict = dict(zip(unique_counts.index, unique_counts.values))
# sorted_cols = sorted(col_dict, key=col_dict.get, reverse=True)
# data = data[sorted_cols]

In [624]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7


In [625]:
# def Partition(input_df, dim, start_row, end_row, dataCount):
#     subset_df = input_df.iloc[start_row:end_row + 1, :]
#     sorted_df = subset_df.sort_values(by=input_df.columns[dim])
#     input_df.iloc[start_row:end_row + 1, :] = sorted_df.values
#     counts = sorted_df.iloc[:, dim].value_counts().sort_index().tolist()
#     dataCount.extend(counts)


In [626]:
import heapq
import os
import openpyxl

def Partition(file_name, dim, start_row, end_row, dataCount):
    workbook = openpyxl.load_workbook(file_name)
    worksheet = workbook.active
    tempFileNames = []

    line_number = 0
    pageSize = 2
    pageIndex = 0
    df = pd.DataFrame()
    for row in worksheet.iter_rows(min_row=start_row+2, max_row=end_row+2, values_only=True):
        if line_number >= pageSize:
            df.sort_values(by=df.columns[dim], inplace=True)
            tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
            df.to_excel(tempFileName, index=False, header=False)
            tempFileNames.append(tempFileName)
            line_number = 0
            pageIndex += 1
            df = pd.DataFrame()

        df = df.append(pd.DataFrame([row]), ignore_index=True)
        line_number += 1

    if line_number > 0:
        print(df.columns[dim])
        df.sort_values(by=df.columns[dim], inplace=True)
        tempFileName = "../data/temporary/" + str(pageIndex) + ".xlsx"
        df.to_excel(tempFileName, index=False, header=False)
        tempFileNames.append(tempFileName)
        pageIndex += 1

    print("done")


    min_heap = []

    # Open all the temporary files
    file_handles = [open(tempFileName, 'r') for tempFileName in tempFileNames]

    # open  our original file
    original_file = open(file_name, 'r+')

    # go to the start row
    original_file.seek(start_row)

    # Initialize the heap with the first row from each file
    for i, file_handle in enumerate(file_handles):
        row = pd.read_excel(file_handle, nrows=1, header=None)
        heapq.heappush(min_heap, (row.iloc[0, dim], i, row))
        file_handle.seek(1, os.SEEK_CUR)

    # while the heap is not empty
    while len(min_heap) > 0:
        value, file_index, row = heapq.heappop(min_heap)
        # clear the first row of the original file
        original_file.write(row.to_string(header=False, index=False))
        original_file.write('\n')
        original_file.seek(1, os.SEEK_CUR)

        dataCountLength = len(dataCount)
        if dataCountLength == 0:
            dataCount.append({
                "value": value,
                "count": 1
            })
        elif dataCount[dataCountLength - 1]["value"] == value:
            dataCount[dataCountLength - 1]["count"] += 1
        else:
            dataCount.append({
                "value": value,
                "count": 1
            })

        # read the next row from the file of index file_index if it is not empty
        row = pd.read_excel(file_handles[file_index], nrows=1, header=None)
        if not row.empty:
            heapq.heappush(min_heap, (row.iloc[0, dim], file_index, row))
            file_handles[file_index].seek(1, os.SEEK_CUR)


    # close all the files
    for file_handle in file_handles:
        file_handle.close()

    original_file.close()

    # delete all the temporary files
    for tempFileName in tempFileNames:
        os.remove(tempFileName)

In [627]:
def BottomUpCube(input, dim, start_row, end_row, currentCombination, numDims, minSup):
    global outputRec
    for d in range(dim, numDims):
        dataCount = {}
        Partition(input, d, start_row, end_row, dataCount)

        k = start_row
        for value, c in dataCount.items():
            if c >= minSup:
                outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{
                    "Combination": currentCombination + [value], 
                    "Count": c
                    }])], ignore_index=True)

                BottomUpCube(input, d + 1, k, k + c - 1, currentCombination + [value], numDims, minSup)

            k += c    

In [628]:
count = len(data)
if count >= minSup:
    outputRec = pd.concat([pd.DataFrame(outputRec), pd.DataFrame([{"Combination": [], "Count": count}])], ignore_index=True)

In [629]:
fileName = "../data/inputs/test.xlsx"

In [630]:
BottomUpCube(fileName, 0, 0, 3, [], numDims, minSup)

0
done


/tmp/ipykernel_8664/29817220.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)
/tmp/ipykernel_8664/29817220.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([row]), ignore_index=True)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 16: invalid start byte

In [ ]:
print(outputRec)

  Combination  Count
0          []    3.0


In [ ]:
data.head()

,A,B,C
0,3,3,7
1,3,3,7
2,2,4,7
